In [1]:
import transformers
from unsloth import FastLanguageModel
from typing import Optional
import os
import torch
import random

model, tokenizer = FastLanguageModel.from_pretrained(
                model_name="unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit",
                max_seq_length=32768,
                fast_inference=True
            )

FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-20 15:56:31 __init__.py:190] Automatically detected platform cuda.
Unsloth: Switching from Unsloth dynamic quant to normal quant since
we do not yet support fast inference for unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit
==((====))==  Unsloth 2025.2.5: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.621 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/DeepSeek-R1-Distill-Qwen-1.5B-bnb-4bit with actual GPU utilization = 25.51%
Unsloth: Your GPU has CUDA compute capability 8.9

[W220 15:56:40.185148841 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 02-20 15:56:41 weight_utils.py:252] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-20 15:56:41 model_runner.py:1115] Loading model weights took 1.5365 GB
INFO 02-20 15:56:41 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-20 15:56:43 worker.py:267] Memory profiling takes 1.42 seconds
INFO 02-20 15:56:43 worker.py:267] the current vLLM instance can use total_gpu_memory (23.62GiB) x gpu_memory_utilization (0.26) = 6.03GiB
INFO 02-20 15:56:43 worker.py:267] model weights take 1.54GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 2.14GiB; the rest of the memory reserved for KV Cache is 2.27GiB.
INFO 02-20 15:56:43 executor_base.py:110] # CUDA blocks: 5311, # CPU blocks: 11702
INFO 02-20 15:56:43 executor_base.py:115] Maximum concurrency for 32768 tokens per request: 2.59x
INFO 02-20 15:56:44 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error oc

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:07<00:00,  3.60it/s]

INFO 02-20 15:56:52 model_runner.py:1562] Graph capturing finished in 8 secs, took 0.54 GiB
INFO 02-20 15:56:52 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 10.47 seconds


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536, padding_idx=151654)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear4bit(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear4bit(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((153

In [5]:
tokenized_prompt = tokenizer.encode(f"<｜begin▁of▁sentence｜><｜User｜>How many r's are in Strawberry?<｜Assistant｜>", return_tensors="pt", add_special_tokens=False).to(model.device)

#generated_text = model.generate(tokenized_prompt, max_new_tokens=100, do_sample=True, temperature=0.6, top_p=0.95)
#print(tokenizer.decode(generated_text[0]))

In [ ]:
# Example parameters:
min_thinking_tokens = 700  # Minimum tokens that must be produced between <think> and <\think>
max_output_tokens = 1000    # Total tokens we want in the final output
max_batch_tokens = 20      # Generate tokens in small batches
replacement_text = "Wait"  # Replacement text if end is premature

# Encode the markers to get their token IDs.
start_think_token = tokenizer.encode("<think>", add_special_tokens=False)[0]
end_think_token = tokenizer.encode("</think>", add_special_tokens=False)[0]
replacement_ids = tokenizer.encode(replacement_text, add_special_tokens=False)

# Initialize flags and counters.
is_thinking = False         # Are we currently in the thinking phase?
thinking_token_count = 0    # Number of tokens produced during thinking (between <think> and <\think>)

# Start with the prompt (already on the model’s device).
final_tokens = tokenized_prompt

# Loop until the final sequence length reaches max_output_tokens.
while final_tokens.shape[1] < max_output_tokens:
    remaining = max_output_tokens - final_tokens.shape[1]
    batch_new_tokens = min(max_batch_tokens, remaining)
    
    outputs = model.generate(
        input_ids=final_tokens,
        max_new_tokens=batch_new_tokens,
        do_sample=True,
        temperature=0.6,
        top_p=0.95,
    )
    # Only the tokens generated in this batch (beyond what we already had)
    new_tokens = outputs[:, final_tokens.shape[1]:]

    for token in new_tokens[0]:
        # If we've reached our overall length limit, stop.
        if final_tokens.shape[1] >= max_output_tokens:
            break

        token_id = token.item()

        # Handle start-of-thinking marker.
        if token_id == start_think_token:
            is_thinking = True
            thinking_token_count = 0
            final_tokens = torch.cat(
                [final_tokens, token.unsqueeze(0).unsqueeze(0)], dim=1
            )
            continue

        # Handle end-of-thinking marker.
        if token_id == end_think_token:
            if is_thinking:
                if thinking_token_count < min_thinking_tokens:
                    # Not enough thinking tokens generated yet:
                    # Determine how many tokens we can inject without exceeding max_output_tokens.
                    remaining_injection = max_output_tokens - final_tokens.shape[1]
                    if len(replacement_ids) > remaining_injection:
                        injection_ids = replacement_ids[:remaining_injection]
                    else:
                        injection_ids = replacement_ids
                    final_tokens = torch.cat(
                        [final_tokens, torch.tensor([injection_ids]).to(final_tokens.device)],
                        dim=1
                    )
                    thinking_token_count += len(injection_ids)
                    # Stay in thinking mode; do not exit it.
                    continue
                else:
                    # Minimum thinking tokens reached; accept the end marker.
                    final_tokens = torch.cat(
                        [final_tokens, token.unsqueeze(0).unsqueeze(0)], dim=1
                    )
                    is_thinking = False
                    continue
            else:
                # If not in thinking mode, just append the token.
                final_tokens = torch.cat(
                    [final_tokens, token.unsqueeze(0).unsqueeze(0)], dim=1
                )
                continue

        # If we're in thinking mode, count the token.
        if is_thinking:
            thinking_token_count += 1

        # Append the current token.
        final_tokens = torch.cat(
            [final_tokens, token.unsqueeze(0).unsqueeze(0)], dim=1
        )
        
        # Stop early if the model outputs the EOS token.
        if token_id == model.config.eos_token_id:
            break

    # Also break out of the loop if EOS was generated.
    if final_tokens[0, -1].item() == model.config.eos_token_id:
        break

output_text = tokenizer.decode(final_tokens[0], skip_special_tokens=True)
print(output_text)

<｜User｜>How many r's are in Strawberry?<｜Assistant｜><think>
I need to determine how many times the letter 'r' appears in the word "Strawberry."

First, I'll write down the word: S T R A W B E R R Y.

Next, I'll go through each letter and count the 'r's.

The first letter is 'S' (no 'r'), the second is 'T' (no 'r'), the third is 'R' (count = 1), the fourth is 'A' (no 'r'), the fifth is 'W' (no 'r'), the sixth is 'B' (no 'r'), the seventh is 'E' (no 'r'), the eighth is 'R' (count = 2), the ninth is 'R' (count =3), and the tenth is 'Y' (no 'r').

So, in total, there are three 'r's in the word "Strawberry."
Wait

To determine how many times the letter 'r' appears in the word "Strawberry," I will go through each letter and count each occurrence of 'r.'

The word is: S, T, R, A, W, B, E, R, R, Y.

Counting the 'r's:

1. The first letter is 'S' – no 'r.'
2. The second letter is 'T' – no 'r.'
3. The third letter is 'R' – that's one 'r.'
4. The fourth letter is 'A' – no 'r.'
5. The fifth letter

In [7]:
# ----------------------------
# Configuration and Parameters
# ----------------------------
max_output_tokens = 1024         # Overall maximum tokens in the output.
batch_new_tokens = 50            # Number of tokens to generate in one batch.
min_thinking_tokens = 500         # Minimum tokens that must be generated in thinking mode.
start_think_token = 151648         # Hypothetical token ID for the start-of-thinking marker.
end_think_token = 151649           # Hypothetical token ID for the end-of-thinking marker.
replacement_text = "Wait"
replacement_ids = tokenizer.encode(replacement_text, add_special_tokens=False)
final_tokens = tokenized_prompt

# ----------------------------
# Generate New Tokens
# ----------------------------
# Generate a batch of new tokens from the model.
outputs = model.generate(
    final_tokens,
    max_new_tokens=batch_new_tokens,
    do_sample=True,
    temperature=0.6,
    top_p=0.95,
)

# Extract only the new tokens (those beyond the already existing ones).
new_tokens = outputs[:, final_tokens.shape[1]:]

# ----------------------------
# Custom Token-Processing Loop
# ----------------------------
# We'll use a phase variable to manage our three modes:
# "normal"   : before any thinking marker is encountered.
# "thinking" : after a start-of-thinking marker and before sufficient tokens have been generated.
# "answer"   : after finishing the thinking phase, waiting for the answer's end marker.
phase = "normal"
thinking_token_count = 0
answer_generated = False

for token in new_tokens[0]:
    # Stop processing if we've reached the overall token limit.
    if final_tokens.shape[1] >= max_output_tokens:
        break

    token_id = token.item()

    # If we see the start-of-thinking token, switch to thinking mode.
    if token_id == start_think_token:
        phase = "thinking"
        thinking_token_count = 0
        final_tokens = torch.cat([final_tokens, token.unsqueeze(0).unsqueeze(0)], dim=1)
        continue

    if phase == "thinking":
        # Intercept the end-of-thinking token.
        if token_id == end_think_token:
            if thinking_token_count < min_thinking_tokens:
                # Not enough thinking tokens have been produced; inject replacement tokens.
                remaining_injection = max_output_tokens - final_tokens.shape[1]
                if len(replacement_ids) > remaining_injection:
                    injection_ids = replacement_ids[:remaining_injection]
                else:
                    injection_ids = replacement_ids
                final_tokens = torch.cat(
                    [final_tokens, torch.tensor([injection_ids]).to(final_tokens.device)],
                    dim=1
                )
                thinking_token_count += len(injection_ids)
                # Stay in thinking mode.
                continue
            else:
                # Enough thinking tokens have been generated; switch to answer phase.
                phase = "answer"
                continue
        else:
            # In thinking mode, count and append tokens.
            thinking_token_count += 1
            final_tokens = torch.cat([final_tokens, token.unsqueeze(0).unsqueeze(0)], dim=1)
            # Optionally, break if the model generates an EOS token.
            if token_id == model.config.eos_token_id:
                break
            continue

    if phase == "answer":
        # In answer phase, ignore tokens until an end-of-thinking token appears.
        if token_id == end_think_token:
            # Append the single answer token and exit.
            final_tokens = torch.cat([final_tokens, token.unsqueeze(0).unsqueeze(0)], dim=1)
            answer_generated = True
            break
        else:
            # Skip any tokens until the end marker is generated.
            continue

    # Normal processing (when not in thinking or answer mode): append token as usual.
    final_tokens = torch.cat([final_tokens, token.unsqueeze(0).unsqueeze(0)], dim=1)
    if token_id == model.config.eos_token_id:
        break

# ----------------------------
# Decode and Output the Result
# ----------------------------
output_text = tokenizer.decode(final_tokens[0], skip_special_tokens=True)
print(output_text)


<｜User｜>How many r's are in Strawberry?<｜Assistant｜><think>
Alright, so I'm trying to figure out how many times the letter 'r' appears in the word "Strawberry." Let me start by writing down the word to make sure I have it correctly. "Strawberry" is
